### Manipulação de Dados da Calculadora de Aluguel - V2
Como tivemos problemas com a conversão dos endereços para coordenadas de latitude e longitude, estudei a API do TomTom Maps https://developer.tomtom.com/content/search-api-explorer

Apesar da limitação de 2500 queries diários gratuitos, a quantidade é suficiente para fazer a conversão dessa base da dados.

In [1]:
#Imports
import pandas as pd
import numpy as np
import requests
import re

Uma mudança que se fez necessária no meu teste para interpretar os dados foi de converter o arquivo .csv para UTF-8

In [2]:
#Parameters for TOMTOM
#I fenced the search on the City of Rio de Janeiro using the topLeft and btmRight coordinates
url = 'https://raw.githubusercontent.com/LuizFelipeAG/awaritasks/main/calculadora_aluguel/manipulacao/dadosrj-utf8.csv'
TOMTOM_KEY='My_TomTom_Key'
TOMTOM_URL='https://api.tomtom.com/search/2/geocode/{}.json?limit=1&countrySet=BR&topLeft=-22.79820%2C%20-43.79587&btmRight=-23.01830%2C%20-43.14903&key={}'

In [3]:
#Functions

def endereco2latlon(endereco):
    tomtom_request_url=TOMTOM_URL.format(endereco, TOMTOM_KEY)
    response = requests.get(tomtom_request_url, timeout=None).json()
    endereco_lat = response['results'][0]['position']['lat']
    endereco_lon = response['results'][0]['position']['lon']
    return (endereco_lat, endereco_lon)

def extrair_latitude(coordenada):
  if coordenada != '':
    lat = float(coordenada.split(',')[0].replace('(', ''))
  else:
    lat = np.nan
  return lat

def extrair_longitude(coordenada):
  if coordenada != '':
    lon = float(coordenada.split(',')[1].replace(')', ''))
  else:
    lon = np.nan
  return lon

def extrair_area(entrada):
  """Criar função que possua como entrada um texto e retorne a área contida no
  mesmo. Retornar np.nan se for ausentes. Alguns exemplos:
      >>> extrair_area('79 m² construídos1 quarto')
      >>> 79
      >>> extrair_area('280 m² construídos')
      >>> 280
      >>> extrair_area('2,600 m² construídos'
      >>> 2600
      >>> extrair_area('2 quartos')
      >>> np.nan
      """
  if ' m²' in entrada:
    area = entrada.split(' m²')[0]
    area = area.replace(',', '')
  else:
    area = np.nan
  return area

def extrair_quartos(entrada):
  """Criar função que possua como entrada um texto e retorne uma tupla com a 
  área e o quarto. Retornar np.nan se um dos dois for ausentes. Alguns exemplos:
      >>> extrair_quarto('79 m² construídos1 quarto')
      >>> 1
      >>> extrair_quarto('280 m² construídos')
      >>> np.nan
      >>> extrair_quarto('2,600 m² construídos'
      >>> np.nan
      >>> extrair_quarto('2 quartos')
      >>> 2"""
  if 'quarto' in entrada:
    n_quartos = re.findall('(\d*) quarto', entrada)[0]
  else:
    n_quartos = np.nan
  return n_quartos

In [4]:
df = pd.read_csv(url, thousands='.', encoding='utf-8')

### Primeira visusalização

In [5]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio De Janeiro Cen...",970
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio De Janeiro Centr...",1224
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio De Janeiro Centro,...",1000
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio De Janeiro Centro, ...",2200
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio De Jan...",1001


In [6]:
df.isna().sum()

zona               0
area_quartos       0
enderecos_lista    0
aluguel            0
dtype: int64

In [7]:
df['zona'].value_counts()

zona-norte    336
centro        336
zona-sul      336
zona-oeste    336
Name: zona, dtype: int64

In [8]:
df.enderecos_lista.value_counts()

Centro, Rio De Janeiro Centro, Rio De Janeiro                                               205
Rua Riachuelo, Centro, Rio De Janeiro Centro, Rio De Janeiro                                 21
Vila Valqueire, Rio De Janeiro Zona Oeste, Rio De Janeiro                                    16
Avenida Nossa Senhora De Copacabana, Copacabana, Rio De Janeiro Zona Sul, Rio De Janeiro     15
Estrada Dos Bandeirantes, Jacarepaguá, Rio De Janeiro Zona Oeste, Rio De Janeiro             13
                                                                                           ... 
Rua Augusto Nunes, Todos Os Santos, Rio De Janeiro Zona Norte, Rio De Janeiro                 1
Rua Major Azevedo, Irajá, Rio De Janeiro Zona Norte, Rio De Janeiro                           1
Rua Tomás Coelho, Vila Isabel, Rio De Janeiro Zona Norte, Rio De Janeiro                      1
Rua Dos Andradas, Centro, Rio De Janeiro Centro, Rio De Janeiro                               1
Rua José Higino, Tijuca, Rio De Janeiro 

### Limpeza
A captura do selenium trouxe repetições no nome do bairro e cidade. Como estamos lidando somente com a cidade do Rio de Janeiro, vamos fazer uma substituição destes dados como limpeza. Isto irá facilitar a captura de coordenadas na API do TomTom.

In [9]:
df['enderecos_lista'].replace('Rio De Janeiro ?(.*)','Rio de Janeiro, RJ', inplace= True, regex = True)

In [10]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio de Janeiro, RJ",970
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio de Janeiro, RJ",1224
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio de Janeiro, RJ",1000
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio de Janeiro, RJ",2200
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio de Jan...",1001


#### Capturando as coordenadas através do TomTom

In [11]:
#Getting Lat and Lon from addresses
df['coordenadas']=df['enderecos_lista'].apply(endereco2latlon).astype(str)

In [12]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel,coordenadas
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio de Janeiro, RJ",970,"(-22.91041, -43.17737)"
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio de Janeiro, RJ",1224,"(-22.9123, -43.17794)"
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio de Janeiro, RJ",1000,"(-22.90892, -43.19369)"
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio de Janeiro, RJ",2200,"(-22.91353, -43.18043)"
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio de Jan...",1001,"(-22.91777, -43.69936)"


In [13]:
#aplicando funcoes para criar as colunas latitude e longitude
df['latitude']=df['coordenadas'].apply(extrair_latitude).astype(float)
df['longitude']=df['coordenadas'].apply(extrair_longitude).astype(float)

In [15]:
# removendo a coluna coordenadas
df = df.drop('coordenadas', axis=1)

#### Separando as colunas área e quartos

In [16]:
# Aplicando funcao para criar a coluna area
df['area']=df['area_quartos'].apply(extrair_area).astype(float)

In [17]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel,latitude,longitude,area
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio de Janeiro, RJ",970,-22.91041,-43.17737,50.0
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio de Janeiro, RJ",1224,-22.91230,-43.17794,28.0
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio de Janeiro, RJ",1000,-22.90892,-43.19369,38.0
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio de Janeiro, RJ",2200,-22.91353,-43.18043,62.0
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio de Jan...",1001,-22.91777,-43.69936,48.0


In [18]:
# Aplicando funcao para criar a coluna quartos
df['quartos'] = df['area_quartos'].apply(extrair_quartos).astype(float)

In [19]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel,latitude,longitude,area,quartos
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio de Janeiro, RJ",970,-22.91041,-43.17737,50.0,1.0
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio de Janeiro, RJ",1224,-22.91230,-43.17794,28.0,1.0
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio de Janeiro, RJ",1000,-22.90892,-43.19369,38.0,1.0
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio de Janeiro, RJ",2200,-22.91353,-43.18043,62.0,2.0
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio de Jan...",1001,-22.91777,-43.69936,48.0,2.0


In [20]:
# removendo a coluna area_quartos
df = df.drop('area_quartos', axis=1)

In [21]:
# reposicionando as colunas
cols = ['zona', 'enderecos_lista','latitude', 'longitude', 'area', 'quartos', 'aluguel']

In [22]:
df=df[cols]

In [23]:
df.head()

,zona,enderecos_lista,latitude,longitude,area,quartos,aluguel
0,centro,"Rua Senador Dantas, Centro, Rio de Janeiro, RJ",-22.91041,-43.17737,50.0,1.0,970
1,centro,"Rua Das Marrecas, Centro, Rio de Janeiro, RJ",-22.91230,-43.17794,28.0,1.0,1224
2,centro,"Rua De Santana, Centro, Rio de Janeiro, RJ",-22.90892,-43.19369,38.0,1.0,1000
3,centro,"Rua Riachuelo, Centro, Rio de Janeiro, RJ",-22.91353,-43.18043,62.0,2.0,2200
4,centro,"Rua João Neves Da Fontoura, Centro, Rio de Jan...",-22.91777,-43.69936,48.0,2.0,1001


In [24]:
# antes de movermos para preprocessamentos voltados à valores ausentes, vamos 
# salvar os dados do jeito como estão
df.to_csv('dados_aluguel_ml_rj_preprocessado.csv', index=False)

In [25]:
# Index=false usado anteriormente para que o csv seja carregado sem necessitar argumentos adicionais
df = pd.read_csv('dados_aluguel_ml_rj_preprocessado.csv')

In [26]:
df.head()

,zona,enderecos_lista,latitude,longitude,area,quartos,aluguel
0,centro,"Rua Senador Dantas, Centro, Rio de Janeiro, RJ",-22.91041,-43.17737,50.0,1.0,970
1,centro,"Rua Das Marrecas, Centro, Rio de Janeiro, RJ",-22.91230,-43.17794,28.0,1.0,1224
2,centro,"Rua De Santana, Centro, Rio de Janeiro, RJ",-22.90892,-43.19369,38.0,1.0,1000
3,centro,"Rua Riachuelo, Centro, Rio de Janeiro, RJ",-22.91353,-43.18043,62.0,2.0,2200
4,centro,"Rua João Neves Da Fontoura, Centro, Rio de Jan...",-22.91777,-43.69936,48.0,2.0,1001
